# 경기 아이디 추출

In [1]:
key = '0TvQnueqKa5mxJntVWt0w4LpLfEkrV1Ta8rQBb9Z'

In [2]:
import requests
header = {'x-api-key':key}

leagues = 'https://esports-api.lolesports.com/persisted/gw/getLeagues?hl=ko-KR'
r = requests.get(leagues,headers = header)
data = r.json()
data

{'data': {'leagues': [{'id': '100695891328981122',
    'image': 'http://static.lolesports.com/leagues/1585044513499_EM_BUG_2020%20(1).png',
    'name': 'European Masters',
    'priority': 215,
    'region': '유럽',
    'slug': 'european-masters'},
   {'id': '101097443346691685',
    'image': 'http://static.lolesports.com/leagues/1592516072459_TAL-01-FullonDark.png',
    'name': 'TAL',
    'priority': 1000,
    'region': '터키',
    'slug': 'turkey-academy-league'},
   {'id': '101382741235120470',
    'image': 'http://static.lolesports.com/leagues/1592516315279_LLA-01-FullonDark.png',
    'name': 'Liga Latinoamérica',
    'priority': 209,
    'region': '라틴 아메리카',
    'slug': 'lla'},
   {'id': '104366947889790212',
    'image': 'http://static.lolesports.com/leagues/1592515942679_PCS-01-FullonDark.png',
    'name': 'PCS',
    'priority': 1000,
    'region': '홍콩, 마카오, 대만',
    'slug': 'pcs'},
   {'id': '98767975604431411',
    'image': 'http://static.lolesports.com/leagues/1592594612171_Worlds

In [6]:

match_list_18 = []
header = {'x-api-key':key}
param = {'leagueId':98767975604431411}

leagues = 'https://esports-api.lolesports.com/persisted/gw/getTournamentsForLeague?hl=ko-KR'
r = requests.get(leagues,headers = header,params = param)
data = r.json()
data

{'data': {'leagues': [{'tournaments': [{'endDate': '2018-11-03',
      'id': '100783238182986407',
      'slug': 'world_championship_2018',
      'startDate': '2018-10-01'},
     {'endDate': '2019-11-10',
      'id': '102804906674705057',
      'slug': 'world_championship_2019',
      'startDate': '2019-10-02'},
     {'endDate': '2020-10-31',
      'id': '104841804583318464',
      'slug': 'worlds_2020',
      'startDate': '2020-09-25'},
     {'endDate': '2017-11-04',
      'id': '98767991971246908',
      'slug': 'world_championship_2017',
      'startDate': '2017-09-22'}]}]}}

In [13]:
#get events(all match in tournament)
header = {'x-api-key':key}
#param = {'tournamentId':102804906674705057}
param = {'tournamentId':98767991971246908}

leagues = 'https://esports-api.lolesports.com/persisted/gw/getCompletedEvents?hl=ko-KR'
r = requests.get(leagues,headers = header,params = param)
data = r.json()

In [14]:
data

{'data': {'schedule': {'events': []}}}

In [9]:
#world championship games list
game_list = []

In [10]:
for i in data['data']['schedule']['events']:
    temp = []
    temp.append(i['startTime'])
    for j in i['games']:
        if len(j['vods']) is not 0:
            temp.append(j['id'])
    game_list.append(temp)

In [11]:
#game_list

In [12]:
multi = []
single = []

In [13]:
for i in range(len(game_list)):
    if len(game_list[i])>2:
        multi.append(game_list[i])
    else:
        single.append(game_list[i])

In [13]:
single.pop(19)

['2019-10-04T17:00:00Z', '102905919916854804']

# 경기 기록 추출

## single game 추출

In [23]:
import pandas as pd
import numpy as np
import time

In [24]:
gamedata = {}

In [16]:
single = list(map(lambda x : [x[0][:-1]+'.00Z',x[1]],single))

In [17]:
single[-1]

['2019-10-20T17:00:00.00Z', '102844401152857762']

In [305]:
#get window
import datetime
header = {'x-api-key':key}



for single_game in single:
    
    timeframe = pd.DataFrame(np.zeros((0,2)))
    timeframe.columns = ['Time','frame']
    
    first = 0
    index = 0
    seconds = 0
    last = 0
    dead = 0
    while dead == 0 :
        
        try:
            if first == 0:
                startT = single_game[0]
                timeT = datetime.datetime.strptime(startT, "%Y-%m-%dT%H:%M:%S.%fZ")
                timeT = timeT - datetime.timedelta(hours=1)
                first +=1
            else:
                if seconds == 0 :
                    timeT = timeT + datetime.timedelta(minutes=1)
                else:
                    timeT = timeT + datetime.timedelta(seconds=10)
            param = {'id':single_game[1],'startingTime':str(timeT.strftime("%Y-%m-%dT%H:%M:%S.%fZ"))}
            
            leagues = 'https://feed.lolesports.com/livestats/v1/window/'+str(param['id'])
            r = requests.get(leagues,headers = header,params = param)
            data = r.json()
            for i in range(len(data['frames'])):
                if len(data['frames'][i]['rfc460Timestamp']) == 20:
                    data['frames'][i]['rfc460Timestamp'] = data['frames'][i]['rfc460Timestamp'][:-1]+'.00Z'

                tempTime = datetime.datetime.strptime(data['frames'][i]['rfc460Timestamp'], "%Y-%m-%dT%H:%M:%S.%fZ")

                check = str(tempTime.hour)+'-'+str(tempTime.minute)+'-'+str(tempTime.second)

                if data['frames'][i]['gameState']=='finished':
                    dead = 1
                    if check in list(timeframe['Time']):
                        timeframe.loc[index-1] = check,data['frames'][i]
                        continue
                
                if  check not in list(timeframe['Time']):
                    timeframe.loc[index] = check,data['frames'][i]
                    index += 1
                
            if seconds == 0:
                timeT = timeT - datetime.timedelta(minutes=1)
                timeframe = pd.DataFrame(np.zeros((0,2)))
                timeframe.columns = ['Time','frame']
                seconds =1
            
        except Exception as e:
            pass
    gamedata[single_game[1]] = timeframe
    print('finish')

finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish


In [ ]:
for i in range(len(gamedata)):
    print(gamedata[single[i][1]]['frame'].iloc[[0]].values[0]['blueTeam']['totalGold'])
    print(gamedata[single[i][1]]['frame'].iloc[[-1]].values[0]['gameState'])

# multidata 추출

In [43]:
gamedata = {}

In [21]:
multi = list(map(lambda x : [x[0][:-1]+'.00Z']+x[1:],multi))

In [34]:
#get window
import datetime
header = {'x-api-key':key}



for multi_game in multi:
    
    first = 0
    for gameIndex in range(1,len(multi_game)):
        timeframe = pd.DataFrame(np.zeros((0,2)))
        timeframe.columns = ['Time','frame']

        index = 0
        seconds = 0
        last = 0
        dead = 0
        while dead == 0 :

            try:
                if first == 0:
                    startT = multi_game[0]
                    timeT = datetime.datetime.strptime(startT, "%Y-%m-%dT%H:%M:%S.%fZ")
                    timeT = timeT - datetime.timedelta(hours=1)
                    first +=1
                else:
                    if seconds == 0 :
                        timeT = timeT + datetime.timedelta(minutes=1)
                    else:
                        timeT = timeT + datetime.timedelta(seconds=10)
                param = {'id':multi_game[gameIndex],'startingTime':str(timeT.strftime("%Y-%m-%dT%H:%M:%S.%fZ"))}

                leagues = 'https://feed.lolesports.com/livestats/v1/window/'+str(param['id'])
                r = requests.get(leagues,headers = header,params = param)
                data = r.json()
                for i in range(len(data['frames'])):
                    if len(data['frames'][i]['rfc460Timestamp']) == 20:
                        data['frames'][i]['rfc460Timestamp'] = data['frames'][i]['rfc460Timestamp'][:-1]+'.00Z'

                    tempTime = datetime.datetime.strptime(data['frames'][i]['rfc460Timestamp'], "%Y-%m-%dT%H:%M:%S.%fZ")

                    check = str(tempTime.hour)+'-'+str(tempTime.minute)+'-'+str(tempTime.second)

                    if data['frames'][i]['gameState']=='finished':
                        second = 0
                        dead = 1
                        if check in list(timeframe['Time']):
                            timeframe.loc[index-1] = check,data['frames'][i]
                            continue

                    if  check not in list(timeframe['Time']):
                        timeframe.loc[index] = check,data['frames'][i]
                        index += 1

                if seconds == 0 and dead == 0:
                    timeT = timeT - datetime.timedelta(minutes=1)
                    timeframe = pd.DataFrame(np.zeros((0,2)))
                    timeframe.columns = ['Time','frame']
                    seconds =1

            except Exception as e:
                pass
        gamedata[multi_game[gameIndex]] = timeframe
        print('finish')

finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish
finish


In [47]:
for i in range(len(multi)):
    #print(multi[i])
    for j in range(1,len(multi[i])):
        print(imsidata[multi[i][j]]['frame'].iloc[[0]].values[0]['blueTeam']['totalGold'])
        print(imsidata[multi[i][j]]['frame'].iloc[[-1]].values[0]['gameState'])

0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished
0
finished


In [48]:
import pickle
 
## Save pickle 임시저장

with open("C:/Users/Gangmin/Desktop/gangmin/capstone/lolDeveloperApi/multigame.pickle","wb") as fw:
    pickle.dump(imsidata, fw)


In [2]:
## Load pickle
import pickle
with open("C:/Users/Gangmin/Desktop/gangmin/capstone/lolDeveloperApi/multigame.pickle","rb") as fr:
    imsidata = pickle.load(fr)

In [ ]:
'''
0~18(19개)
19 빼먹음
20~40(21개)


'''

In [ ]:
#2개 마저 수집하고 정확성 확인하기

# gamelist train,test

In [247]:
gamelist = []

In [254]:
for i in data['data']['schedule']['events']:
    print(i)
    break
    for j in i['games']:
        if len(j['vods']) is not 0:
            temp = []
            temp.append(i['startTime']) 
            temp.append(i['match']['teams'][0]['code'])
            temp.append(i['match']['teams'][1]['code'])
            temp.append("_"+str(j['id']))
            gamelist.append(temp)

{'startTime': '2019-10-02T11:00:00Z', 'blockName': '플레이-인 그룹 스테이지', 'league': {'name': '월드 챔피언십'}, 'match': {'id': '102844212420310203', 'type': 'normal', 'teams': [{'name': 'Clutch Gaming', 'code': 'CLU', 'image': 'https://lolstatic-a.akamaihd.net/esports-assets/production/team/clutch-gaming-83a00582.png', 'result': {'gameWins': 0}}, {'name': 'Unicorns of Love', 'code': 'UOL', 'image': 'https://lolstatic-a.akamaihd.net/esports-assets/production/team/unicorns-of-love-8qvakeja.png', 'result': {'gameWins': 1}}], 'strategy': {'type': 'bestOf', 'count': 1}}, 'games': [{'id': '102844212430075086', 'vods': [{'parameter': 'J5R7PrKMBv8'}, {'parameter': 'e3Zi11S2IqE'}, {'parameter': 'U24z9YbJV10'}, {'parameter': 'N7HSF1yukk8'}]}]}


In [249]:
#gamelist

In [250]:
gamelist = pd.DataFrame(gamelist)

In [251]:
gamelist.columns = ['Time','T1','T2','gameId']

In [252]:
import numpy as np
from sklearn.model_selection import train_test_split


X_train, X_test = train_test_split(gamelist, test_size=0.3, random_state=123)


In [253]:
X_train.to_csv("c:/users/gangmin/desktop/gangmin/capstone/lolDeveloperApi/trainlist.csv")
X_test.to_csv("c:/users/gangmin/desktop/gangmin/capstone/lolDeveloperApi/testlist.csv")